In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download atharv1610/terrain-recognition

 99% 3.14G/3.16G [00:16<00:00, 270MB/s]
100% 3.16G/3.16G [00:16<00:00, 203MB/s]


In [ ]:
! unzip terrain-recognition.zip

Streaming output truncated to the last 5000 lines.
  inflating: Data Main/val/Grassy/Grassy (9)_17_0.png  
  inflating: Data Main/val/Grassy/Grassy (9)_17_13.png  
  inflating: Data Main/val/Grassy/Grassy (9)_17_15.png  
  inflating: Data Main/val/Grassy/Grassy (9)_17_19.png  
  inflating: Data Main/val/Grassy/Grassy (9)_17_21.png  
  inflating: Data Main/val/Grassy/Grassy (9)_17_22.png  
  inflating: Data Main/val/Grassy/Grassy (9)_17_23.png  
  inflating: Data Main/val/Grassy/Grassy (9)_17_25.png  
  inflating: Data Main/val/Grassy/Grassy (9)_17_27.png  
  inflating: Data Main/val/Grassy/Grassy (9)_17_31.png  
  inflating: Data Main/val/Grassy/Grassy (9)_17_6.png  
  inflating: Data Main/val/Grassy/Grassy (9)_17_7.png  
  inflating: Data Main/val/Grassy/Grassy (9)_18_15.png  
  inflating: Data Main/val/Grassy/Grassy (9)_18_17.png  
  inflating: Data Main/val/Grassy/Grassy (9)_18_20.png  
  inflating: Data Main/val/Grassy/Grassy (9)_18_24.png  
  inflating: Data Main/val/Grassy/Grassy

In [ ]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
#Dataloader

#Path for training and testing directory
train_path='/content/data/train'
test_path='/content/data/test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(classes)

['Grassy', 'Marshy', 'Rocky', 'Sandy']


In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()

        #Output size after convolution filter
        #((w-f+2P)/s) +1

        #Input shape= (256,3,150,150)

        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)

        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)


        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)



        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)


        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)



        #Feed forwad function

    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)

        output=self.pool(output)

        output=self.conv2(output)
        output=self.relu2(output)

        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)


            #Above output will be in matrix form, with shape (256,32,75,75)

        output=output.view(-1,32*75*75)


        output=self.fc(output)

        return output

In [ ]:
model=ConvNet(num_classes=4)

In [ ]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs=10

In [ ]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.png'))
test_count=len(glob.glob(test_path+'/**/*.png'))

In [ ]:
print(train_count,test_count)

31571 6769


In [ ]:
best_accuracy=0.0

for epoch in range(num_epochs):

    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0

    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        optimizer.zero_grad()

        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()


        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)

        train_accuracy+=int(torch.sum(prediction==labels.data))

    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count


    # Evaluation on testing dataset
    model.eval()

    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/test_count


    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(2.4898) Train Accuracy: 0.7456209812802889 Test Accuracy: 0.8057320135913725
Epoch: 1 Train Loss: tensor(0.6016) Train Accuracy: 0.8284818345950398 Test Accuracy: 0.7376274191165608
Epoch: 2 Train Loss: tensor(1.1413) Train Accuracy: 0.7758385860441545 Test Accuracy: 0.7432412468606885
Epoch: 3 Train Loss: tensor(0.7248) Train Accuracy: 0.7712457635171518 Test Accuracy: 0.7760378194711184
Epoch: 4 Train Loss: tensor(1.7367) Train Accuracy: 0.7083399322162744 Test Accuracy: 0.7373319544984488
Epoch: 5 Train Loss: tensor(0.8397) Train Accuracy: 0.7305121788983561 Test Accuracy: 0.7597872654749593
Epoch: 6 Train Loss: tensor(0.6383) Train Accuracy: 0.7664628931614457 Test Accuracy: 0.7859358841778697
Epoch: 7 Train Loss: tensor(0.5396) Train Accuracy: 0.7947800196382756 Test Accuracy: 0.7633328408923031
Epoch: 8 Train Loss: tensor(0.5474) Train Accuracy: 0.8109657597161952 Test Accuracy: 0.8244940168414833
Epoch: 9 Train Loss: tensor(0.5489) Train Accuracy: 0.8